In [6]:
import glob
import os
import pandas as pd
from datetime import datetime


# Read Maxim location for all the months

In [ ]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = 'C:/Users/nastj/Downloads/Full Data Set/'

csv_files = glob.glob(path + "/*/*naamsestraat-35-maxim.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_maxim_df = pd.concat(df_list, ignore_index=True)
full_maxim_df.shape

del df_list

In [70]:
full_maxim_df.sort_values(by=['result_timestamp'],inplace=True)
full_maxim_df = full_maxim_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_maxim_df.reset_index(inplace=True)
full_maxim_df.to_parquet('Dataset/full_maxim_df.parquet')

In [3]:
maxim_df = pd.read_parquet('Dataset/full_maxim_df.parquet')
maxim_df

,result_timestamp,lamax,laeq,lceq,lcpeak
0,2022-02-28 08:20:00,58.400905,56.715493,67.821278,79.327515
1,2022-02-28 08:40:00,59.563057,57.756369,65.896815,77.866975
2,2022-02-28 09:00:00,NaN,NaN,NaN,NaN
3,2022-02-28 09:20:00,NaN,NaN,NaN,NaN
4,2022-02-28 09:40:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...
21401,2022-12-22 14:00:00,55.792917,53.752500,62.573792,75.210100
21402,2022-12-22 14:20:00,55.173917,53.262000,61.611900,74.158667
21403,2022-12-22 14:40:00,56.143417,54.098583,62.389817,75.100867
21404,2022-12-22 15:00:00,56.173750,54.176333,61.712333,74.699358


selecting night hours + Adding day of the week and holidays

In [4]:
maxim_df = maxim_df.loc[(maxim_df['result_timestamp'].dt.hour <= 7) | (maxim_df['result_timestamp'].dt.hour >= 19)]

maxim_df.loc[:, 'night_scale'] = (maxim_df['result_timestamp'] - pd.Timedelta(hours=8)).dt.strftime('%d-%m-%Y %H:%M')
maxim_df['night_scale'] = pd.to_datetime(maxim_df['night_scale'])
   #night from monday to tuesday counted as monday
maxim_df.loc[:, 'night_hour'] = (maxim_df['night_scale'].dt.hour + maxim_df['night_scale'].dt.minute/60) - 11
maxim_df.loc[:, 'night'] = maxim_df['night_scale'].dt.strftime('%d-%m-%Y')

maxim_df["weekday"] = maxim_df['night_scale'].dt.day_name()
holiday = ["01-01-2022", "18-04-2022", "16-05-2022", "21-07-2022", "25-08-2022", "01-11-2022", "02-11-2022","11-11-2022", "25-12-2022"]
is_holiday = maxim_df['night'].isin(holiday)
maxim_df['Holiday']=is_holiday.map({True: 1, False:0})


C:\Users\katri\AppData\Local\Temp\ipykernel_29228\2508151511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maxim_df.loc[:, 'night_scale'] = (maxim_df['result_timestamp'] - pd.Timedelta(hours=8)).dt.strftime('%d-%m-%Y %H:%M')
C:\Users\katri\AppData\Local\Temp\ipykernel_29228\2508151511.py:4: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  maxim_df['night_scale'] = pd.to_datetime(maxim_df['night_scale'])
C:\Users\katri\AppData\Local\Temp\ipykernel_29228\2508151511.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [40]:
maxim_df['result_timestamp'].dt.month.unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [9]:
missing_df = maxim_df[maxim_df.isnull().any(axis=1)]

In [10]:
missing_df["result_timestamp"].dt.month.unique()

array([ 2,  3, 10, 11, 12])

In [17]:
missing_df['result_timestamp'].dt.day.unique()

array([28, 27, 25, 26, 29, 30, 31,  1,  2, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23,  3,  4,  5,  6,  7,  8,  9])

In [ ]:
import calendar

# missing_df['result_timestamp'].map(lambda x: datetime.datetime(x.month, max(calendar.monthcalendar(x.year, x.month)[-1][:5])))

missing_days_months = missing_df['result_timestamp'].apply(lambda x: x.strftime('%B-%d-´%H:%M')) 
for (date_time) in missing_days_months:
    if datetime.strptime(date_time,'%B-%d-´%H:%M').hour <= 5 or datetime.strptime(date_time,'%B-%d-´%H:%M').hour >=19:
        print(date_time)

In [16]:
missing_df['result_timestamp'].dt.month.unique()

array([ 2,  3, 10, 11, 12])

In [47]:
oct_df = maxim_df[maxim_df['result_timestamp'].dt.month == 10]

In [49]:
missing_oct_df = oct_df[oct_df.isnull().any(axis=1)]

In [ ]:
missing_oct_df

# read Xior location

In [ ]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = 'C:/Users/nastj/Downloads/Full Data Set/'

csv_files = glob.glob(path + "/*/*xior.csv") 
df_list = (pd.read_csv(file, sep=';',
                     usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser)  
                       for file in csv_files)

full_xior_df = pd.concat(df_list, ignore_index=True)
full_xior_df.shape

del df_list

From above code: error dtype float
Below: alternative

In [72]:
path = 'C:/Users/nastj/Downloads/Full Data Set/'

df_list = (pd.read_csv(file, sep = ";")
           for file in glob.glob(path + "/*/*xior.csv") )
full_xior_df = pd.concat(df_list, ignore_index=True)
full_xior_df = full_xior_df[['result_timestamp','lamax','laeq','lceq','lcpeak']]

In [73]:
full_xior_df['result_timestamp'] = full_xior_df['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

full_xior_df.sort_values(by=['result_timestamp'],inplace=True)
full_xior_df = full_xior_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_xior_df.reset_index(inplace=True)
full_xior_df.to_parquet('Dataset/full_xior_df.parquet')

In [54]:
pd.read_parquet('Dataset/full_xior_df.parquet')

# read Taste location

In [ ]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = 'C:/Users/nastj/Downloads/Full Data Set/'

csv_files = glob.glob(path + "/*/*taste.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_taste_df = pd.concat(df_list, ignore_index=True)

In [67]:
full_taste_df.sort_values(by=['result_timestamp'],inplace=True)
full_taste_df = full_taste_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_taste_df.reset_index(inplace=True)
full_taste_df.to_parquet('Dataset/full_taste_df.parquet')

Alternative

In [74]:
path = 'C:/Users/nastj/Downloads/Full Data Set/'

df_list = (pd.read_csv(file, sep = ";")
           for file in glob.glob(path + "/*/*taste.csv") )
full_taste_df = pd.concat(df_list, ignore_index=True)
full_taste_df = full_taste_df[['result_timestamp','lamax','laeq','lceq','lcpeak']]


full_taste_df['result_timestamp'] = full_taste_df['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))
full_taste_df.sort_values(by=['result_timestamp'],inplace=True)
full_taste_df = full_taste_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_taste_df.reset_index(inplace=True)
full_taste_df.to_parquet('Dataset/full_taste_df.parquet')

In [ ]:
full_taste_df = pd.read_parquet('Dataset/full_maxim_df.parquet')
full_taste_df.head()

# read Kapel location

In [ ]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = 'C:/Users/nastj/Downloads/Full Data Set/'

csv_files = glob.glob(path + "/*/*calvariekapel-ku-leuven.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_kapel_df = pd.concat(df_list, ignore_index=True)

In [68]:
full_kapel_df.sort_values(by=['result_timestamp'],inplace=True)
full_kapel_df = full_kapel_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_kapel_df.reset_index(inplace=True)
full_kapel_df.to_parquet('Dataset/full_kapel_df.parquet')

Alternative

In [ ]:
path = 'C:/Users/nastj/Downloads/Full Data Set/'

df_list = (pd.read_csv(file, sep = ";")
           for file in glob.glob(path + "/*/*calvariekapel-ku-leuven.csv") )
full_kapel_df = pd.concat(df_list, ignore_index=True)
full_kapel_df = full_kapel_df[['result_timestamp','lamax','laeq','lceq','lcpeak']]


full_kapel_df['result_timestamp'] = full_kapel_df['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

full_kapel_df.sort_values(by=['result_timestamp'],inplace=True)
full_kapel_df =full_kapel_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_kapel_df.reset_index(inplace=True)
full_kapel_df.to_parquet('Dataset/full_kapel_df.parquet')

# read filosovia location

In [62]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*filosovia.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_filo_df = pd.concat(df_list, ignore_index=True)

In [69]:
full_filo_df.sort_values(by=['result_timestamp'],inplace=True)
full_filo_df = full_filo_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_filo_df.reset_index(inplace=True)
full_filo_df.to_parquet('Dataset/full_filo_df.parquet')

Alternative

In [ ]:
path = 'C:/Users/nastj/Downloads/Full Data Set/'

df_list = (pd.read_csv(file, sep = ";")
           for file in glob.glob(path + "/*/*filosovia.csv"))
full_filo_df = pd.concat(df_list, ignore_index=True)
full_filo_df = full_filo_df[['result_timestamp','lamax','laeq','lceq','lcpeak']]

full_filo_df['result_timestamp'] =full_filo_df['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

full_filo_df.sort_values(by=['result_timestamp'],inplace=True)
full_filo_df =full_filo_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_filo_df.reset_index(inplace=True)
full_filo_df.to_parquet('Dataset/full_filo_df.parquet')

# read 81 location

In [63]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*81.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_81_df = pd.concat(df_list, ignore_index=True)

In [70]:
full_81_df.sort_values(by=['result_timestamp'],inplace=True)
full_81_df = full_81_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_81_df.reset_index(inplace=True)
full_81_df.to_parquet('Dataset/full_81_df.parquet')

Alternative

In [ ]:
path = 'C:/Users/nastj/Downloads/Full Data Set/'

df_list = (pd.read_csv(file, sep = ";")
           for file in glob.glob(path + "/*/*81.csv"))
full_81_df = pd.concat(df_list, ignore_index=True)
full_81_df = full_81_df[['result_timestamp','lamax','laeq','lceq','lcpeak']]

full_81_df['result_timestamp'] =full_81_df['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

full_81_df.sort_values(by=['result_timestamp'],inplace=True)
full_81_df =full_81_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_81_df.reset_index(inplace=True)
full_81_df.to_parquet('Dataset/full_81_df.parquet')

# read kiosk location useless

In [64]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*stadspark.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_kiosk_df = pd.concat(df_list, ignore_index=True)

In [72]:
#full_kiosk_df.sort_values(by=['result_timestamp'],inplace=True)
#full_kiosk_df = full_kiosk_df.resample('15min',closed = 'right', on='result_timestamp').mean()
#full_kiosk_df.reset_index(inplace=True)
full_kiosk_df.to_parquet('Dataset/full_kiosk_df.parquet')

# read hof location

In [65]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*vrijthof.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_hof_df = pd.concat(df_list, ignore_index=True)

In [73]:
full_hof_df.sort_values(by=['result_timestamp'],inplace=True)
full_hof_df = full_hof_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_hof_df.reset_index(inplace=True)
full_hof_df.to_parquet('Dataset/full_hof_df.parquet')

alternative

In [ ]:
path = 'C:/Users/nastj/Downloads/Full Data Set/'

df_list = (pd.read_csv(file, sep = ";")
           for file in glob.glob(path + "/*/*vrijthof.csv"))
full_hof_df = pd.concat(df_list, ignore_index=True)
full_hof_df = full_hof_df[['result_timestamp','lamax','laeq','lceq','lcpeak']]

full_hof_df['result_timestamp'] =full_hof_df['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

full_hof_df.sort_values(by=['result_timestamp'],inplace=True)
full_hof_df =full_hof_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_hof_df.reset_index(inplace=True)
full_hof_df.to_parquet('Dataset/full_hof_df.parquet')

# read his his-hears location

In [66]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*hears.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_hears_df = pd.concat(df_list, ignore_index=True)

In [74]:
full_hears_df.sort_values(by=['result_timestamp'],inplace=True)
full_hears_df = full_hears_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_hears_df.reset_index(inplace=True)
full_hears_df.to_parquet('Dataset/full_hears_df.parquet')

Alternative

In [ ]:
path = 'C:/Users/nastj/Downloads/Full Data Set/'

df_list = (pd.read_csv(file, sep = ";")
           for file in glob.glob(path + "/*/*hears.csv") )
full_hears_df = pd.concat(df_list, ignore_index=True)
full_hears_df = full_hears_df[['result_timestamp','lamax','laeq','lceq','lcpeak']]

full_hears_df['result_timestamp'] =full_hears_df['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

full_hears_df.sort_values(by=['result_timestamp'],inplace=True)
full_hears_df =full_hears_df.resample('20min',closed = 'right', on='result_timestamp').mean()
full_hears_df.reset_index(inplace=True)
full_hears_df.to_parquet('Dataset/full_hears_df.parquet')

Pipeline

In [13]:
from sklearn.pipeline import Pipeline
import pandas as pd
#TODO: improve, get X number of characters from CSV files

def createDataFiles( file_path_end):
    '''
    Loops over the provided file path ends & retrieves csv files to create parquet files 
    '''
    path = 'C:/Users/nastj/Downloads/Full Data Set/'
    
    df_list = (pd.read_csv(file, sep = ";")
    for file in glob.glob(path + "/*/*"+f'{file_path_end}'+".csv") )
    df_name = pd.concat(df_list, ignore_index=True)
    df_name = df_name[['result_timestamp','lamax','laeq','lceq','lcpeak']]

    df_name['result_timestamp'] =df_name['result_timestamp'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

    df_name.sort_values(by=['result_timestamp'],inplace=True)
    df_name =df_name.resample('20min',closed = 'right', on='result_timestamp').mean()
    df_name.reset_index(inplace=True)
    df_name.to_parquet(f"Dataset/full_{file_path_end}_df.parquet")
        
    return df_name
    # df_name = df

df_taste = createDataFiles( file_path_end="taste")
# df_vrijthof = createDataFiles(file_path_end="vrijthof")
df_hears= createDataFiles(file_path_end="hears")
df_kiosk= createDataFiles(file_path_end="stadspark")
df_81= createDataFiles(file_path_end="81")
df_filosovia= createDataFiles(file_path_end="filosovia")
# df_maxim =createDataFiles(file_path_end="maxim")
# df_xior = createDataFiles(file_path_end="xior")
df_kapel = createDataFiles(file_path_end="calvariekapel-ku-leuven")


Pipeline
used for creating new columns, dealing with the missing values 

In [ ]:
# test commit